In [26]:
pip install google-generativeai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/student/.virtualenvs/final-project/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import zipfile
import os
import json
import google.generativeai as genai

In [28]:
with open(r"src/api_keys.json") as f:
        api_keys = json.load(f)

In [29]:
pip install openai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/student/.virtualenvs/final-project/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
from openai import AzureOpenAI

params = ['endpoint', 'deployment', 'subscription_key', 'api_version']
for p in params:
    if p not in api_keys:
        raise ValueError(f"Missing '{p}' in api_keys.json")

endpoint = api_keys["endpoint"]
deployment = api_keys["deployment"]
subscription_key = api_keys["subscription_key"]
api_version = api_keys["api_version"]

def ask_gpt(question, options):
    prompt = f"Question: {question}\n"
    for idx, opt in enumerate(options):
        if opt.strip():
            prompt += f"{chr(ord('A')+idx)}. {opt}\n"
    prompt += "Answer with the letter (A, B, C, or D) of the correct option only."
    print(prompt)
    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=subscription_key,
    )

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=deployment,
        temperature=0,
    )
    answer = response.choices[0].message.content.strip().upper()
    #print(response.choices[0].message.content)
    #print("ChatGPT response:\n", answer)  # Debug print
    for letter in ["A", "B", "C", "D"]:
        if answer.startswith(letter):
            return letter
    return "A"  # fallback


In [31]:
def process_qa_file(filepath):
    correct = 0
    total = 0
    with open(filepath, 'r', encoding='utf-8-sig', errors='replace') as f:
        for line in f:
            if line.strip():
                qa = json.loads(line)
                model_answer = ask_gpt(qa['question'], qa['options'])
                print(f"Q: {qa['question']}\nModel answer: {model_answer}, Correct answer: {qa['correct_answer']}\n")
                if model_answer == qa['correct_answer']:
                    correct += 1
                total += 1
    return correct, total

In [ ]:
import time
zip_path = 'Q&A.zip'
extract_dir = 'extracted_qa'

# Unzip files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Extracted files:", os.listdir(extract_dir))

results = {}
subdir = os.path.join(extract_dir, 'Q&A', 'Q&A')
print("Files in subdir:", os.listdir(subdir))
for fname in os.listdir(subdir):
    fpath = os.path.join(subdir, fname)
    print(f"Checking file: {fpath}")  # Debug print
    if os.path.isfile(fpath) and fname.endswith('.jsonl'):
        print(f"Processing JSON file: {fname}")  # Debug print
        correct, total = process_qa_file(fpath)
        print(f"Result for {fname}: {correct} correct out of {total}")  # Debug print
        results[fname] = {'correct': correct, 'total': total}
        #print("Waiting a minute to respect quota...")
        #time.sleep(60)  # Wait 60 seconds before next file
    else:
        print(f"Skipped: {fname}")  # Debug print

with open("baseline_results.json", 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)
